# Slogan Generator - Test

## 0) Import Dependencies

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import unidecode
import pandas as pd
import string
import json
import requests
import random
import re

## 1) Prepare Data

In [2]:
all_characters = string.printable
n_characters = len(all_characters)

file = unidecode.unidecode(open('../Data/zero_to_one.txt').read())
file_len = len(file)
print('file_len = ', file_len)

file_len =  83354


In [3]:
slogans = []

with open('../Data/zero_to_one.txt') as text_file:
    for line in text_file:
        slogans.append(line.split('\n')[0])

slogans[:10]

['EVERY MOMENT IN BUSINESS happens only once. The next Bill Gates will not build an operating system. The next',
 'Larry Page or Sergey Brin won’t make a search engine. And the next Mark Zuckerberg won’t create a',
 'social network. If you are copying t',
 'Blake Masters, took detailed class notes, which circulated far beyond the campus, and in Zero to One',
 'I have worked with him to revise the notes for a wider audience. There’s no reason why the future',
 'should happen only at Stanford, or in college, or in Silicon Valley.',
 '1',
 'THE CHALLENGE OF THE FUTURE',
 'WHENEVER I INTERVIEW someone for a job, I like to ask this question: “What important truth do very few',
 'people agree with you on?”']

### Picking a Random Slogan

In [4]:
def random_slogan():
    return random.choice(slogans).replace('"', "'")

(random_slogan())

'various exciting elements in entirely new ways.” It could even be true. Suppose her idea is to have'

### *ALTERNATE OPTION* Sample Random Chunks

In [5]:
chunk_len = 200

def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    chunk = file[start_index:end_index]
    return chunk

print(random_chunk())

group of hackers to catch the
shark that killed his friend. That has definitely never been done before. But, like the lack of British
restaurants in Palo Alto, maybe that's a good thing.
The problem wi


## 2) Build the Model

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return (torch.zeros(self.n_layers, 1, self.hidden_size))

### Setting Inputs & Targets

In [7]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return (tensor)

print(char_tensor('abcDEF'))

tensor([ 10,  11,  12,  39,  40,  41])


We can assemble a pair of input and target tensors for training, from a random chunk. The input will be all characters up to the last, and the target will be all characters from the first.

**NOTE** Because *random_slogan()* returns string with single quotes, ```'str'```, we need to convert it into a double quoted string. That way, we don't encounter issues with strings that contain commas.

There is no way in Python to do this, but by using **json**, we can output a double quoted string from the argument we pass in

In [8]:
def random_training_set():    
    chunk = json.dumps(random_chunk())
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## 3) Set up the Evaluator

In [9]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str
    
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
        inp = prime_input[-1]
        
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)
        
    return predicted

## 4) Training

First, we will set up a helper function to keep track of time during training

In [10]:
import time, math

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

Now, for the main training function:

In [11]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden)
        loss += criterion(output, target[c].unsqueeze(0))

    loss.backward()
    decoder_optimizer.step()

    return loss.data.item() / chunk_len

We define our hyperparameters, instantiate the model, and start straining:

In [ ]:
n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 256
n_layers = 3
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    loss = train(*random_training_set())       
    loss_avg += loss
    
#     print("Time this takes: %s" % (time_since(start)))

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print(evaluate('Wh', 100), '\n')

    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

 44s (100 5%) 2.4529]
Whe xidinade,. In t\"nde this Pon oy or'd. Ind ad woao and in the nald the an inn in the cord os bay h 

 29s (200 10%) 2.1375]
Wh_er a cont the jard Irind gingat ong awed tho and dour dand ind of veome conne ant yopde redive the  

 16s (300 15%) 2.1533]
Wh) endet the afuching wa thongeroney--thed and non tivable cunttas 29999997 andere 2999O undetion. I 

 0s (400 20%) 1.9853]
Wh thoughhgan soluil grame you dow eavial the fater on ged that the envan k0ome at moneal co-up in man 

 44s (500 25%) 1.9082]
Whe it worke revertibuing witte plilizated innite prout the you miller p promics Vlived of the youve y 

 25s (600 30%) 2.1291]
Whhe a lot's ar and caper, the produuse. Fod perete's to conome, talle. Cone to vicises lo one and all 



### Plotting the Training Loss

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

plt.figure()
plt.plot(all_losses)

## 5) Evaluation: Creating Potential Slogans

In [ ]:
print(evaluate('Hello', 100, temperature=0.8))

In [ ]:
print(evaluate('Zero ', 100, temperature=0.3))

In [ ]:
print(evaluate('Th', 10, temperature=1.4))

In [ ]:
def get_evaluations(num_predictions=100, predict_len=20, temperature=0.4):
    samples = []
    
    while len(samples) != num_predictions:
        prime_str = random.choice(string.ascii_uppercase)
        samples.append(evaluate(prime_str, predict_len, temperature))
        
    return samples

## 6) Sentiment Analysis on Potential Slogans

In [ ]:
def get_sentiment(string):
    r = requests.post("http://text-processing.com/api/sentiment/", data={'text': string})
    return json.loads(r.text)

In [ ]:
def get_candidates(num_candidates=5, predict_len=20, temperature=0.8):
    candidates = []
    # Keep track of how many evaluations we calculate
    sentiment_evaluations = 0
    # Evaluate potential candidated until we have our desired amount
    while len(candidates) != num_candidates:
        prime_str = random.choice(string.ascii_uppercase)
        sample = evaluate(prime_str, predict_len, temperature)
        
        # With predicted sample, run through sentiment analysis
        sentiment = get_sentiment(sample)
        sentiment_evaluations += 1
        
        # Finalizing candidate if it has a strong enough score
        if sentiment['probability']['pos'] > 0.65:
            print(sentiment['probability'])
            candidates.append(sample)
            
        # Stop early if calculate too many evaluations (request limit)
        if sentiment_evaluations >= 1000:
            print("too many attempts: " + str(sentiment_evaluations))
            return candidates
        
    print("number of sentiment evaluations done: " + str(sentiment_evaluations))
    return candidates

In [ ]:
get_candidates()

## FINAL: Exporting the Trained Model

For use outside of this notebook, we want to save the model and make it easy to import in other files

In [ ]:
torch.save(decoder.state_dict(), '../MVP/rnn.py')